In [ ]:
import torch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers_rnn = 2
num_layers_lstm = 3
num_layers_gru = 10
num_classes = 10
batch_size = 50
num_epochs = 5
learning_rate = 0.001

In [ ]:
import torchvision
import torchvision.transforms as transforms

In [ ]:
train_data = torchvision.datasets.MNIST(root='./datasets',
                                        train=True,
                                        transform=transforms.ToTensor(),
                                        download=True)
test_data = torchvision.datasets.MNIST(root='./datasets',
                                        train=False,
                                        transform=transforms.ToTensor(),
                                        download=True)

Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw


Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/raw




Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw


Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
image, label = next(iter(test_loader))
print(image.size())

torch.Size([50, 1, 28, 28])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

### RNN

In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, nonlinearity = 'relu')
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    
    out, _ = self.rnn(x, h0.detach())

    out = self.fc(out[:,-1,:])

    return out

### LSTM

In [ ]:
class LSTM(nn.Module):
  def __init__(self, intput_size, hidden_size, num_layers, num_classes):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 

    out, _  = self.lstm(x, (h0.detach(), c0.detach()))

    out = self.fc(out[:,-1,:])

    return out

### GRU

In [ ]:
class GRU(nn.Module):
  def __init__(self, intput_size, hidden_size, num_layers, num_classes):
    super(GRU, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    out, _  = self.gru(x, h0.detach())

    out = self.fc(out[:,-1,:])

    return out

#### Training

In [ ]:
def train(model):
  total_step = len(train_loader)
  for epoch in range(num_epochs):
    for i, (image, label) in enumerate(train_loader):
      image = image.reshape(-1, sequence_length, input_size).to(device)
      label = label.to(device)

    # Forward
      output = model(image) 
      loss = criterion(output, label)

    # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (i+1) % 400 == 0:
        print("Epoch [{}/{}], Step[{}/{}], Loss:{:.4f}".format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# train RNN
model = RNN(input_size, hidden_size, num_layers_rnn, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train(model)
torch.save(model.state_dict(), 'rnn_20170154.pth')

Epoch [1/5], Step[400/1200], Loss:0.5055
Epoch [1/5], Step[800/1200], Loss:0.3453
Epoch [1/5], Step[1200/1200], Loss:0.2092
Epoch [2/5], Step[400/1200], Loss:0.3730
Epoch [2/5], Step[800/1200], Loss:0.2650
Epoch [2/5], Step[1200/1200], Loss:0.0616
Epoch [3/5], Step[400/1200], Loss:0.0425
Epoch [3/5], Step[800/1200], Loss:0.0866
Epoch [3/5], Step[1200/1200], Loss:0.1271
Epoch [4/5], Step[400/1200], Loss:0.2223
Epoch [4/5], Step[800/1200], Loss:0.1673
Epoch [4/5], Step[1200/1200], Loss:0.1959
Epoch [5/5], Step[400/1200], Loss:0.0901
Epoch [5/5], Step[800/1200], Loss:0.0746
Epoch [5/5], Step[1200/1200], Loss:0.0618


In [ ]:
# train LSTM
model = LSTM(input_size, hidden_size, num_layers_lstm, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train(model)
torch.save(model.state_dict(), 'lstm_20170154.pth')

Epoch [1/5], Step[400/1200], Loss:0.2866
Epoch [1/5], Step[800/1200], Loss:0.0707
Epoch [1/5], Step[1200/1200], Loss:0.1106
Epoch [2/5], Step[400/1200], Loss:0.2574
Epoch [2/5], Step[800/1200], Loss:0.1033
Epoch [2/5], Step[1200/1200], Loss:0.0084
Epoch [3/5], Step[400/1200], Loss:0.0102
Epoch [3/5], Step[800/1200], Loss:0.0255
Epoch [3/5], Step[1200/1200], Loss:0.0895
Epoch [4/5], Step[400/1200], Loss:0.0120
Epoch [4/5], Step[800/1200], Loss:0.0416
Epoch [4/5], Step[1200/1200], Loss:0.0126
Epoch [5/5], Step[400/1200], Loss:0.0287
Epoch [5/5], Step[800/1200], Loss:0.1281
Epoch [5/5], Step[1200/1200], Loss:0.0315


In [ ]:
# train GRU
model = GRU(input_size, hidden_size, num_layers_gru, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train(model)
torch.save(model.state_dict(), 'gru_20170154.pth')

Epoch [1/5], Step[400/1200], Loss:0.3404
Epoch [1/5], Step[800/1200], Loss:0.4191
Epoch [1/5], Step[1200/1200], Loss:0.0632
Epoch [2/5], Step[400/1200], Loss:0.1567
Epoch [2/5], Step[800/1200], Loss:0.0889
Epoch [2/5], Step[1200/1200], Loss:0.0583
Epoch [3/5], Step[400/1200], Loss:0.0125
Epoch [3/5], Step[800/1200], Loss:0.0176
Epoch [3/5], Step[1200/1200], Loss:0.1629
Epoch [4/5], Step[400/1200], Loss:0.0314
Epoch [4/5], Step[800/1200], Loss:0.0395
Epoch [4/5], Step[1200/1200], Loss:0.0706
Epoch [5/5], Step[400/1200], Loss:0.0410
Epoch [5/5], Step[800/1200], Loss:0.0086
Epoch [5/5], Step[1200/1200], Loss:0.0727


#### Loading & Testing Accuracy

In [ ]:
# load RNN model
model = RNN(input_size, hidden_size, num_layers_rnn, num_classes).to(device)
model.load_state_dict(torch.load('rnn_20170154.pth'), strict=False)

with torch.no_grad():
  correct = 0

  for image, label in test_loader:
    image = image.reshape(-1, sequence_length, input_size).to(device)
    label = label.to(device)
    output = model(image)
    _, pred = torch.max(output.data, 1)
    correct += (pred == label).sum().item()

  print('Test Accuracy of RNN model on the {} test images: {}%'.format(len(test_data), 100 * correct / len(test_data)))

Test Accuracy of RNN model on the 10000 test images: 97.5%


In [ ]:
# load LSTM model
model = LSTM(input_size, hidden_size, num_layers_lstm, num_classes).to(device)
model.load_state_dict(torch.load('lstm_20170154.pth'), strict=False)

with torch.no_grad():
  correct = 0

  for image, label in test_loader:
    image = image.reshape(-1, sequence_length, input_size).to(device)
    label = label.to(device)
    output = model(image)
    _, pred = torch.max(output.data, 1)
    correct += (pred == label).sum().item()

  print('Test Accuracy of LSTM model on the {} test images: {}%'.format(len(test_data), 100 * correct / len(test_data)))

Test Accuracy of LSTM model on the 10000 test images: 98.23%


In [ ]:
# load GRU model
model = GRU(input_size, hidden_size, num_layers_gru, num_classes).to(device)
model.load_state_dict(torch.load('gru_20170154.pth'), strict=False)

with torch.no_grad():
  correct = 0

  for image, label in test_loader:
    image = image.reshape(-1, sequence_length, input_size).to(device)
    label = label.to(device)
    output = model(image)
    _, pred = torch.max(output.data, 1)
    correct += (pred == label).sum().item()

  print('Test Accuracy of GRU model on the {} test images: {}%'.format(len(test_data), 100 * correct / len(test_data)))

Test Accuracy of GRU model on the 10000 test images: 97.96%
